In [12]:
import pyDOE
import pandas as pd
import numpy as np
import Utils 
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import Bounds
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple
from matplotlib import cm
import cma
from scipy.optimize import minimize
from scipy.optimize import Bounds
import sys

## Helper Functions

In [13]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' Plot the Graph of the Function'''
def plot_the_graph(x,y,z,graph_title):
    fig = plt.figure(figsize=(12,5))
    ax = Axes3D(fig)
    surf = ax.plot_trisurf(x,y,z,cmap=cm.jet, linewidth=0.3,alpha=0.9)
    fig.colorbar(surf, shrink=10, aspect=3)
    ax.set_xlabel('X1')
    ax.set_ylabel('X2')
    ax.set_zlabel('Robust(X1,X2)')
    plt.title(graph_title)
    plt.show()
    fig.savefig(graph_title+'.pdf')

''' Original Function for Optimization '''
def ackley_2D(X):
    x1,x2 = X
    A = (-20 *  np.exp (-0.2 * np.sqrt(( (x1**2) + (x2**2) ) / (2) ) ) )
    B = np.exp (((np.cos(2 * np.pi * x1 ) ) + (np.cos(2 * np.pi * x2 ) ) ) / (2) )
    return A - B +20 +np.exp(1)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=2, samples=n_obs, criterion='m')
    X1, X2 = lhd[:,0], lhd[:,1]
    return X1,X2


'''Robust Regularization based on minimax Principle 5% perturbations'''
def robust_regularization(X):
    x1,x2 = X
    eps = np.linspace(-6.5536,6.5536,2000)
    return np.max( ackley_2D([x1+eps,x2+eps]) )

''' Robustness based on a composite function of Mean and STd '''
def composite_robustness(X):
    w=1
    x1,x2 = X
    np.random.seed(0)
    eps = np.random.normal(loc=0.0,scale=1.0922666666666667,size=10000)
    sample_points = ackley_2D([x1+eps,x2+eps])
    sample_mean = np.mean(sample_points)
    variance = np.square(sample_points-sample_mean)
    std = np.sqrt(np.mean(variance))
    std = np.sqrt(np.mean(variance))
    return sample_mean + w * std

''' Generate Training Data using LHD along side the Output of the Robust System'''
def generate_training_data(n_obs):
    X1,X2 = DOE(n_obs)
    X1 = linearscaletransform(X1,range_out=(-32.768,32.768))
    X2 = linearscaletransform(X2,range_out=(-32.768,32.768))
    f_evaluation = ackley_2D([X1,X2])
    f_original = np.zeros(X1.shape[0])
    minimax_original = np.zeros(X1.shape[0])
    composite_original = np.zeros(X1.shape[0])
    for i in range(X1.shape[0]):
        f_original[i] = ackley_2D([X1[i],X2[i]])
        minimax_original[i] = robust_regularization([X1[i],X2[i]])
        composite_original[i] = composite_robustness([X1[i],X2[i]])
    train = pd.DataFrame()
    train['X1'] = pd.Series(X1)
    train['X2'] = pd.Series(X2)
    train['Y = F(X1,X2)'] = pd.Series(f_original)
    train['Y = robust_regularization(X1,X2)'] = pd.Series(minimax_original)
    train['Y = composite(X1,X2)'] = pd.Series(composite_original)
    train.to_csv('Training_Data_Sets\\'+str(n_obs)+'Samples.csv')
    return train

''' Generate Test Data using LHD along side the Output of the Robust System'''
def generate_test_data(n_obs):
    test = pd.DataFrame()
    np.random.seed(0)
    lhd = pyDOE.lhs(n=2, samples=n_obs, criterion='m')
    X1 = linearscaletransform(lhd[:,0],range_out=(-32.768,32.768))
    X2 = linearscaletransform(lhd[:,1],range_out=(-32.768,32.768))
    test ['X1'] = pd.Series(X1)
    test ['X2'] = pd.Series(X2)
    true_minimax = np.zeros(test.shape[0])
    true_composite = np.zeros(test.shape[0])
    for i in range(test.shape[0]):
        true_minimax[i] = robust_regularization(test.iloc[i,:])
        true_composite[i] = composite_robustness(test.iloc[i,:])
    test ['True_Minimax'] = pd.Series(true_minimax)
    test ['True_Composite'] = pd.Series(true_composite)
    test.to_csv('Test_Data_Sets\\'+str(n_obs)+'Samples.csv')
    return test

''' Check if There are same examples in Test and Train Data Sets'''
def check_data_sets(train,test):
    C = train.isin(test)
    C = pd.DataFrame.sum(C,axis=1)
    ids = C.index[C>1]
    print ('Total Number of Input Observations present in Test Data : '+str(ids.shape[0]))
